In [1]:
!pip install -U sentence-transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=4d24a951b288486b88f062b89487f66b00c55316003b84add97804cdb70daadb
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
from sentence_transformers import SentenceTransformer, losses, InputExample
from torch.utils.data import DataLoader
from datasets import load_dataset

model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [3]:
# Sentences we want to encode. Example:
question = ['What color is the Sky?']
answers = ["the sky is blue", "I ate eggs for breakfast","what color is the sea?","How high is the sky?"]

# Sentences are encoded by calling model.encode()
embedding_q = model.encode(question)
embedding_a = model.encode(answers)

In [4]:
len(embedding_q), len(embedding_a)

(1, 4)

In [5]:
import numpy as np

Q = embedding_q[0]
#for ans_vec in embedding_a:
for i in range(len(embedding_a)):
  A = embedding_a[i]
  ans = answers[i]
  dot_product = np.dot(Q, A)
  print(ans + " --- " + str(dot_product))



the sky is blue --- 0.60815
I ate eggs for breakfast --- 0.13605711
what color is the sea? --- 0.7040468
How high is the sky? --- 0.6920786


## Fine Tune

In [6]:
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
dataset = dataset.filter(lambda example: example['category'] == "closed_qa")

Filter:   0%|          | 0/15011 [00:00<?, ? examples/s]

In [8]:
dataset = dataset.train_test_split(test_size=0.3)

In [9]:
len(dataset["train"]), len(dataset["test"])

(1241, 532)

In [10]:
train_examples = []

for row in dataset["train"]:
  i = InputExample(texts=[ row["instruction"],  row["context"]  ]) #training the model to find the right contexts
  #i = InputExample(texts=[ row["instruction"],  row["response"]  ])
  train_examples.append(i)



In [11]:

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.MultipleNegativesRankingLoss(model=model) #use if you hav esentence pairs
#train_loss = losses.TripletLoss(model=model)  # use this if you mand achor, positive, negative triplets

In [12]:
num_epochs =  10
warmup_steps = 1

In [13]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/78 [00:00<?, ?it/s]

Iteration:   0%|          | 0/78 [00:00<?, ?it/s]

Iteration:   0%|          | 0/78 [00:00<?, ?it/s]

Iteration:   0%|          | 0/78 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/78 [00:00<?, ?it/s]

Iteration:   0%|          | 0/78 [00:00<?, ?it/s]

Iteration:   0%|          | 0/78 [00:00<?, ?it/s]

Iteration:   0%|          | 0/78 [00:00<?, ?it/s]

Iteration:   0%|          | 0/78 [00:00<?, ?it/s]

In [14]:
import pickle

# Save the model
with open('sentence_transformer_model.pkl', 'wb') as f:
    pickle.dump(model, f)

## Load saved model and retest

In [15]:
with open('sentence_transformer_model.pkl', 'rb') as f:
    finetuned_model = pickle.load(f)

In [16]:
base_model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')

In [17]:
def compare_qna(question, answers):
  # Sentences are encoded by calling model.encode()
  embedding_q_finetuned  = finetuned_model.encode(question)
  embedding_a_finetuned = finetuned_model.encode(answers)
  embedding_q_pre_finetuned  = base_model.encode(question)
  embedding_a_pre_finetuned = base_model.encode(answers)
  print("QUESTION:" + question[0] )
  print('ANSWERS: ')
  Q1 = embedding_q_finetuned[0]
  Q2 = embedding_q_pre_finetuned[0]
  #for ans_vec in embedding_a:
  for i in range(len(answers)):
    A1 = embedding_a_finetuned[i]
    A2 = embedding_a_pre_finetuned[i]
    ans = answers[i]
    dot_product1 = np.dot(Q1, A1)
    dot_product2 = np.dot(Q2, A2)
    print(ans + " --SIMILARITY NOW--> " + str(dot_product1) + " --IT WAS--> " + str(dot_product2))

In [18]:
# Sentences we want to encode. Example:
question = ['What color is the Sky?']
answers = ["the sky is blue", "I ate eggs for breakfast","what color is the sea?","How high is the sky?"]

compare_qna(question, answers)

QUESTION:What color is the Sky?
ANSWERS: 
the sky is blue --SIMILARITY NOW--> 0.7082328 --IT WAS--> 0.60815
I ate eggs for breakfast --SIMILARITY NOW--> 0.07757717 --IT WAS--> 0.13605711
what color is the sea? --SIMILARITY NOW--> 0.6535989 --IT WAS--> 0.7040468
How high is the sky? --SIMILARITY NOW--> 0.65446347 --IT WAS--> 0.6920786


In [19]:
# Sentences we want to encode. Example:
question = ['When was Virat Kohli born?']
answers = ["Virat Kohli was born on 5 November 1988 in Delhi","Virat Kohli was born on 5 November 1988","Virat Kohli was born in Delhi", 'When was Sachin Tendulkar born?', "Sachin Tendulkar was born on  Apr 24, 1973"]

compare_qna(question, answers)

QUESTION:When was Virat Kohli born?
ANSWERS: 
Virat Kohli was born on 5 November 1988 in Delhi --SIMILARITY NOW--> 0.93281174 --IT WAS--> 0.8464414
Virat Kohli was born on 5 November 1988 --SIMILARITY NOW--> 0.9474531 --IT WAS--> 0.8673086
Virat Kohli was born in Delhi --SIMILARITY NOW--> 0.8652167 --IT WAS--> 0.8180146
When was Sachin Tendulkar born? --SIMILARITY NOW--> 0.5642234 --IT WAS--> 0.73575497
Sachin Tendulkar was born on  Apr 24, 1973 --SIMILARITY NOW--> 0.52651346 --IT WAS--> 0.55869246


## Evaluate the Test set for Cosine Similarity

In [20]:
num_tests = len(dataset["test"])
#num_tests = 100
num_tests

532

In [21]:
qa_cos_fine_tuned = []
qa_cos_base = []

for i in range(num_tests):
  q = dataset["test"][i]["instruction"]
  a = dataset["test"][i]["context"]
  #print(q)
  #print(a)

  embedding_q_finetuned  = finetuned_model.encode(q)
  embedding_a_finetuned = finetuned_model.encode(a)
  qa_cos_fine_tuned.append(np.dot(embedding_q_finetuned, embedding_a_finetuned))

  embedding_q_pre_finetuned  = base_model.encode(q)
  embedding_a_pre_finetuned = base_model.encode(a)
  qa_cos_base.append(np.dot(embedding_q_pre_finetuned, embedding_a_pre_finetuned))


In [22]:
def Average(lst):
  return str(sum(lst) / len(lst))

In [23]:
len(qa_cos_base)

532

In [24]:
print("Avg. Q&A Cosine Similarity before Fine Tuning: " + Average(qa_cos_base))
print("Avg. Q&A Cosine Similarity after Fine Tuning: " + Average(qa_cos_fine_tuned))

Avg. Q&A Cosine Similarity before Fine Tuning: 0.621808123256647
Avg. Q&A Cosine Similarity after Fine Tuning: 0.6991952516950835


## Accuracy Evaluation (not a great test as the model was fairly accurate to begin with)

In [25]:
#Evaluate Accuracy
def evaluate_accuracy(model):
  questions = []
  answers = []

  for row in dataset["test"]:
    questions.append(row["instruction"])
    answer_tup = (row["context"], model.encode(row["context"]))
    answers.append(answer_tup )

  score = len(questions)
  q_iter = 0
  a_iter = 0
  for q in questions:
    question_emb = model.encode(q)
    q_iter = q_iter + 1
    a_iter = 0
    max_ans_ind = 0
    max_sim = 0
    for ans, ans_emb in answers:
      a_iter = a_iter + 1
      #print(np.dot(question_emb, ans_emb))
      sim = np.dot(question_emb, ans_emb)
      if sim > max_sim:
        max_sim = sim
        max_ans_ind = a_iter
    if (max_ans_ind != q_iter):
      score = score -1

    #break;
    return(score)


In [26]:
s = evaluate_accuracy(finetuned_model)
s

532

In [27]:
s = evaluate_accuracy(base_model)
s

532